Challenge 

The challenge is to generate one right sequence through Genetic Programming. The sequence has to be 8 numbers between 0 to 7. Each number represents the positions the Queens can be placed. Each number refers to the row number in the specific column 

In [102]:
class Eight_Queens():
    '''takes nQueens,mutationRate,totalPopulation,crossOver as an arguments'''
    def __init__(self,nQueens,mutationRate,totalPopulation,crossOver):
        self.nQueens = nQueens
        self.mutationRate = mutationRate
        self.totalPopulation = totalPopulation
        self.crossOver = crossOver
        
    def generateChromosome(self):
        init_distribution = np.arange(self.nQueens)
        np.random.shuffle(init_distribution)
        return init_distribution
    
    def generate_population(self):
        '''function to generate population'''
        my_population = []
        for i in range(self.totalPopulation):
            population = self.generateChromosome()
            my_population.append(population)
        return my_population
    
    def fitness(self,chromosome):
        '''function to caluculate fitness'''
        clashes = 0;
        row_col_clashes = abs(len(chromosome) - len(np.unique(chromosome)))
        clashes += row_col_clashes
        
        for i in range(len(chromosome)):
            for j in range(len(chromosome)):
                if ( i != j):
                    dx = abs(i-j)
                    dy = abs(chromosome[i] - chromosome[j])
                    if(dx == dy):
                        clashes += 1

        return 28 - clashes
    
    def population_data_frame(self):
        pop_fitness = []
        proba_data = []
        my_population = self.generate_population()
        for chromozome in my_population:
            fitness_score = self.fitness(chromozome)
            pop_fitness.append(fitness_score)
        for i in range(len(my_population)):
            prob = pop_fitness[i]/8
            proba_data.append(prob)
        df = pd.DataFrame({'population' : my_population, 'fitness' : pop_fitness, 'probality' : proba_data})
        df = df.sort_values(['probality'],ascending=False)
        
        return df
    
    def mutation_flag(self):
        mutate = [False] * int(100 - self.mutation_rate *100 )
        mutate.extend([True] * int(self.mutation_rate * 100))
        
        return random.choice(mutate)
    
    def view_element(self,data):
        
        return data

    
    def crossover_and_mutation(self):
        '''function for crossover and mutation'''
        crossOverPoint = int(self.crossOver*self.nQueens)
        generationCount = 1000
        my_population = self.generate_population()
        

        for gen in range(generationCount):
            if gen == 0:
                df = self.population_data_frame()
            if gen % 50 ==0:
                df = df.reset_index(drop=True)
            draw=[]
            draw.append(df[0:1]["population"].values[0])
            draw.append(df[1:2]["population"].values[0])

            if self.fitness(draw[0]) == 28:
                print('generation: ', gen, 'fitness_score: ', self.fitness(draw[0]),'Element: ',self.view_element(draw[0]))
                break
            child11 = draw[0][:crossOverPoint]
            chils12 = draw[1][crossOverPoint:]
            child_1 = np.concatenate((child11,chils12),axis=None)
            child_1
            child21 = draw[1][:crossOverPoint]
            child22 = draw[0][crossOverPoint:]
            child_2 = np.concatenate((child21,child22),axis=None)   
            child_1[random.randint(0,self.nQueens-1)] = random.randint(0,7)
            child_2[random.randint(0,self.nQueens-1)] = random.randint(0,7)
            x_list = draw[0].copy()
            x_list = x_list.tolist()
            y_list = draw[0].copy()
            index = []
            for i in range(len(x_list)):
                for j in range(len(x_list)):
                    if ( i != j):
                        dx = abs(i-j)
                        dy = abs(x_list[i] - x_list[j])
                        if(dx == dy):
                            index.extend([j])

            values = []
            for i in index:
                values.extend([x_list[i]])
                
#             random.shuffle(values)
            np.put(draw[0],index,values[::-1])
            child_3 = draw[0].copy()
            np.put(y_list,index,np.random.choice(8,len(index),replace = False))
            child_4 = y_list.copy()
            if gen % 10 == 0:
                child_3[random.randint(0,self.nQueens-1)] = random.randint(0,7)
            if gen % 10 == 0:
                child_4[random.randint(0,self.nQueens-1)] = random.randint(0,7)
            child_5 = draw[0].copy()
            
            unique_elements, counts_elements = np.unique(draw[0], return_counts=True)
            c = -1
            for i in counts_elements:
                if i > 1:
                    child_51 = draw[c][:crossOverPoint]
                    child_52 = draw[c-1][crossOverPoint:]
                    child_5 = np.concatenate((child_51,child_52),axis=None)
                    c -= 1
                    counts_elements = []
                    break 
            child_5[random.randint(0,self.nQueens-1)] = random.randint(0,7)
            my_population.append(child_1)
            my_population.append(child_2)
            my_population.append(child_3)
            my_population.append(child_4)
            my_population.append(child_5)
            
            pop_fitness = []
            proba_data = []
            for chromozome in my_population:
                fitness_score = self.fitness(chromozome)
                pop_fitness.append(fitness_score)
            for i in range(len(my_population)):
                prob = pop_fitness[i]/8
                proba_data.append(prob)
            df = pd.DataFrame({'population' : my_population, 'fitness' : pop_fitness, 'probality' : proba_data})
            df = df.sort_values(['probality'],ascending=False)
#             print(df.head())
            print('Generation', gen, 'Average_fitness', sum(pop_fitness)/len(pop_fitness), 'c1', child_1, 'c2', child_2, 'c3', child_3 )


In [105]:
if __name__ == '__main__':
    import numpy as np
    import random
    import pandas as pd
    my = Eight_Queens(8,0.1,500,0.5)
    my.crossover_and_mutation()

Generation 0 Average_fitness 18.265346534653464 c1 [6 5 6 2 7 1 6 0] c2 [4 2 3 5 3 7 1 4] c3 [3 6 5 2 0 7 2 4]
Generation 1 Average_fitness 18.27843137254902 c1 [2 5 7 3 7 5 1 5] c2 [0 3 6 2 0 4 4 1] c3 [2 3 7 5 0 6 4 1]
Generation 2 Average_fitness 18.293203883495146 c1 [0 3 6 2 6 5 2 5] c2 [5 7 1 3 7 4 1 5] c3 [0 3 6 4 7 2 1 5]
Generation 3 Average_fitness 18.29423076923077 c1 [7 2 0 6 4 0 0 7] c2 [6 2 5 1 4 1 2 3] c3 [7 2 0 6 4 3 5 1]
Generation 4 Average_fitness 18.30666666666667 c1 [6 1 2 4 1 3 1 7] c2 [1 2 5 6 7 0 3 5] c3 [6 2 1 4 7 0 3 5]
Generation 5 Average_fitness 18.324528301886794 c1 [6 2 6 1 6 4 1 7] c2 [3 0 0 2 4 0 3 7] c3 [6 2 5 1 7 0 3 4]
Generation 6 Average_fitness 18.327102803738317 c1 [4 7 1 7 4 2 5 6] c2 [1 3 0 1 6 0 2 5] c3 [4 2 1 3 6 0 7 5]
Generation 7 Average_fitness 18.32777777777778 c1 [0 2 5 6 4 2 5 6] c2 [1 3 2 7 1 3 0 7] c3 [4 2 6 5 1 3 0 7]
Generation 8 Average_fitness 18.332110091743118 c1 [3 6 0 2 4 2 5 6] c2 [1 3 0 7 6 6 1 7] c3 [3 5 0 4 6 2 1 7]
Gener

Generation 74 Average_fitness 20.645714285714284 c1 [5 5 6 1 6 4 0 7] c2 [5 2 6 1 6 4 0 7] c3 [5 2 6 1 6 4 0 7]
Generation 75 Average_fitness 20.673863636363638 c1 [5 2 6 5 7 4 0 7] c2 [5 2 6 1 6 4 1 7] c3 [5 2 6 1 6 4 0 7]
Generation 76 Average_fitness 20.701694915254237 c1 [5 3 6 1 6 4 0 7] c2 [5 2 6 1 7 7 0 7] c3 [5 2 6 1 7 4 0 7]
Generation 77 Average_fitness 20.724719101123597 c1 [5 2 6 0 6 4 0 7] c2 [5 2 6 1 6 4 6 7] c3 [5 2 6 1 6 4 0 7]
Generation 78 Average_fitness 20.748603351955307 c1 [5 5 6 1 6 4 0 7] c2 [5 4 6 1 6 4 0 7] c3 [5 2 6 1 6 4 0 7]
Generation 79 Average_fitness 20.77111111111111 c1 [3 2 6 1 7 4 0 7] c2 [4 2 6 1 6 4 0 7] c3 [5 2 6 1 6 4 0 7]
Generation 80 Average_fitness 20.788950276243092 c1 [5 2 6 1 6 4 0 0] c2 [5 2 6 7 6 4 0 7] c3 [5 2 6 1 6 4 0 6]
Generation 81 Average_fitness 20.80989010989011 c1 [5 2 6 1 6 3 0 7] c2 [2 2 6 1 6 4 0 7] c3 [5 2 6 1 6 4 0 7]
Generation 82 Average_fitness 20.83497267759563 c1 [2 2 6 1 6 4 0 7] c2 [5 2 6 5 7 4 0 7] c3 [5 2 6 1 7 4 